<a href="https://colab.research.google.com/github/poomshift/PromptAlchemist_SD_Webui_Colab/blob/main/PromptAlchemist_SD_Webui_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Prompt Alchemist’s SD Colab

📢 พบปัญหาการใช้งาน กรุณาแจ้งมาที่ [Facebook/promptalchemist](https://www.facebook.com/PromptAlchemist)

⚠️ Google Colab Free ไม่สามารถใช้งานได้อีกต่อไป

📝 สมัคร Google Colab Pro [คลิกที่นี่](https://colab.research.google.com/signup)


> ราคา 343฿ ต่อเดือน ได้รับ 100 computing units


> การใช้งานประมาณ 1.96 units/hr = ชั่วโมงละ 6.7฿

📃 [คู่มือการใช้งานอย่างละเอียด](https://promptalchemist.notion.site/Prompt-Alchemist-s-SD-Colab-3873d42a38db4251ab9f9842206dc22f?pvs=4)

📃 [วิธีลงโมเดลเพิ่มเติม](https://www.notion.so/promptalchemist/Prompt-Alchemist-s-SD-Colab-3873d42a38db4251ab9f9842206dc22f?pvs=4#330e33ab2a58440c9e9f69b37115c090)

ส่วนเสริม:
> | Batchlinks| Tagcomplete | Image-browsing | Controlnet | Adetailer |

> | ReActor | NSFW_Censor | StyleSelectorXL | Animatediff | Photopea |

> | Civitai-Helper | Openpose-editor |

ถ้าต้องกาารใช้โมเดลจาก Google Drive ให้สร้างโฟเดอร์ใน Gdrive

"**Checkpoint**"  สำหรับใส่โมเดลเช็คพ้อย

"**Lora**" สำหรับใส่ Lora

📢 พื้นที่ประกาศกิจกรรม

> The Ultimate Guide to Graphic Design with AI รุ่นที่ 4 จัดโดย Skooldio

> วันที่ 18 และ 25 พฤษภาคม 2024  [>>สนใจคลิก<<](https://www.skooldio.com/workshops/ai-art-graphic-design-bundle?skuCode=P00078-04)

บรรยาย In-house สนใจติดต่อ [Facebook/promptalchemist](https://www.facebook.com/PromptAlchemist)

In [ ]:
# @title Run ที่นี่  { form-width: "30%" }
from IPython.display import Javascript

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'''))
from IPython.display import clear_output, display, HTML
import os
import time
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from urllib.parse import unquote
from google.colab.output import eval_js
import shutil

%cd /content
drive.mount('/content/drive')

%env TF_CPP_MIN_LOG_LEVEL=1
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2
#!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 torchdata torchtext  --index-url https://download.pytorch.org/whl/cu121 -U
!pip install -q xformers
!pip install httpx==0.24.0
!pip install gradio --upgrade
!pip uninstall wandb -y
!pip install wandb==0.15.12

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/8x_NMKD-Superscale_150000_G.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 8x_NMKD-Superscale_150000_G.pth
!git clone https://github.com/camenduru/tunnels /content/stable-diffusion-webui/extensions/tunnels
!git clone https://github.com/etherealxx/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete   /content/stable-diffusion-webui/extensions/a1111-sd-webui-tagcomplete
!pip install basicsr
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui/extensions/ultimateSD
!git clone https://github.com/richrobber2/canvas-zoom /content/stable-diffusion-webui/extensions/canvas-zoom
!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing /content/stable-diffusion-webui/extensions/sd-webui-infinite-image-browsing
!git clone https://github.com/yankooliveira/sd-webui-photopea-embed /content/stable-diffusion-webui/extensions/sd-webui-photopea-embed
#git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper /content/stable-diffusion-webui/extensions/Stable-Diffusion-Webui-Civitai-Helper
!git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/depth-lib
!git clone https://github.com/BlafKing/sd-civitai-browser-plus /content/stable-diffusion-webui/extensions/sd-civitai-browser-plus
# @markdown ส่วนเสริม
Controlnet = True # @param {type:"boolean"}
if Controlnet == True:
  !git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
else:
  ()
Controlnet_models = 'canny, depth, normal, mlsd, openpose, lineart, inpaint, scribble, softedge, tile, shuffle, ip2p, qrcode, ip-adapter' # @param ["canny, depth, normal, mlsd, openpose, lineart, inpaint, scribble, softedge, tile, shuffle, ip2p, qrcode, ip-adapter"] {allow-input: true}

Adetailer = False # @param {type:"boolean"}
if Adetailer == True:
  !git clone https://github.com/Bing-su/adetailer /content/stable-diffusion-webui/extensions/adetailer
else:
  ()

ReActor = False # @param {type:"boolean"}
if ReActor == True:
  !git clone https://github.com/Gourieff/sd-webui-reactor /content/stable-diffusion-webui/extensions/sd-webui-reactor
  !pip install insightface==0.7.3
else:
  ()

#Animatediff = False # @param {type:"boolean"}
#if Animatediff == True:
 # !git clone https://github.com/continue-revolution/sd-webui-animatediff /content/stable-diffusion-webui/extensions/sd-webui-animatediff
  #motion module
#  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/159987 -d /content/stable-diffusion-webui/extensions/sd-webui-animatediff/model
#  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-animatediff/model -o mm_sd15_v3.safetensors
#else:
#  ()

#NSFW_Censor = False # @param {type:"boolean"}
#if NSFW_Censor == True:
#  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-nsfw-censor /content/stable-diffusion-webui/extensions/stable-diffusion-webui-nsfw-censo
#else:
#  ()


Multidiffusion = False # @param {type:"boolean"}
if Multidiffusion == True:
  !git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /content/stable-diffusion-webui/extensions/Multidiffusion
else:
  ()

#Openpose_editor = False # @param {type:"boolean"}
#if Openpose_editor == True:
#  !git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
#else:
#  ()

#StyleSelectorXL = False # @param {type:"boolean"}
#if StyleSelectorXL == True:
#  !git clone https://github.com/ahgsql/StyleSelectorXL.git /content/stable-diffusion-webui/extensions/StyleSelectorXL
#  !pip install insightface==0.7.3
#else:
#  ()

%cd /content/stable-diffusion-webui

# Define the file path and content
file_path = '/content/stable-diffusion-webui/config.json'
config_content = '''
{
  "outdir_samples": "",
  "outdir_txt2img_samples": "/content/drive/MyDrive/SD-IMG-OUT/outputs/text",
  "outdir_img2img_samples": "/content/drive/MyDrive/SD-IMG-OUT/outputs/image",
  "outdir_extras_samples": "/content/drive/MyDrive/SD-IMG-OUT/outputs/extras",
  "outdir_grids": "",
  "outdir_txt2img_grids": "/content/drive/MyDrive/SD-IMG-OUT/outputs/grids",
  "outdir_img2img_grids": "/content/drive/MyDrive/SD-IMG-OUT/outputs/grids",
  "outdir_save": "/content/drive/MyDrive/SD-IMG-OUT/outputs/save",
  "outdir_init_images": "/content/drive/MyDrive/SD-IMG-OUT/outputs/init-images"
}
'''
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Write the config file
with open(file_path, 'w') as config_file:
    config_file.write(config_content)

!git reset --hard
!git -C /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard

%cd /content/stable-diffusion-webui

#controlnet model
if Controlnet == True:
  #canny
  if Controlnet_models.find("canny") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.yaml
  else:
    ()
  #depth
  if Controlnet_models.find("depth") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.yaml
  else:
    ()
  #normal
  if Controlnet_models.find("normal") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.yaml
  else:
    ()
  #mlsd
  if Controlnet_models.find("mlsd") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.yaml
  else:
    ()
   #openpose
  if Controlnet_models.find("openpose") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.yaml
  else:
    ()
   #lineart
  if Controlnet_models.find("lineart") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.yaml
  else:
    ()
   #inpaint
  if Controlnet_models.find("inpaint") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.yaml
  else:
    ()
   #scribble
  if Controlnet_models.find("scribble") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_scribble_fp16.yaml
  else:
    ()
   #softedge
  if Controlnet_models.find("softedge") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.yaml
  else:
    ()
   #tile
  if Controlnet_models.find("tile") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1e_sd15_tile_fp16.yaml
  else:
    ()
   #shuffle
  if Controlnet_models.find("shuffle") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.yaml
  else:
    ()
   #ip2p
  if Controlnet_models.find("ip2p") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.yaml
  else:
    ()
   #qrcode
  if Controlnet_models.find("qrcode") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v1p_sd15_qrcode_monster.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v1p_sd15_qrcode_monster.yaml
  else:
    ()
   #ip-adapter
  if Controlnet_models.find("ip-adapter") != -1:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o ip-adapter_sd15.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o ip-adapter-plus-face_sd15.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o ip-adapter-plus_sd15.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o ip-adapter-faceid-plus_sd15.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors -d /content/stable-diffusion-webui/models/Lora -o ip-adapter-faceid-plus_sd15_lora.safetensors
    !pip install insightface
  else:
    ()
else:
  ()


# @markdown Checkpoint
Checkpoint_01 = "https://civitai.com/api/download/models/176425" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {Checkpoint_01} -d /content/stable-diffusion-webui/models/Stable-diffusion
Checkpoint_02 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {Checkpoint_02} -d /content/stable-diffusion-webui/models/Stable-diffusion
Checkpoint_03 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {Checkpoint_03} -d /content/stable-diffusion-webui/models/Stable-diffusion
# @markdown LoRA
LoRA_01 = "https://civitai.com/api/download/models/132876" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_01} -d /content/stable-diffusion-webui/models/Lora
LoRA_02 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_02} -d /content/stable-diffusion-webui/models/Lora
LoRA_03 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_03} -d /content/stable-diffusion-webui/models/Lora
LoRA_04 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_04} -d /content/stable-diffusion-webui/models/Lora

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/cyberdelia/CyberRealisticClassic/resolve/main/CyberRealisticCLASSIC_V1.7_FP16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o CyberRealisticCLASSIC_V1.7_FP16.safetensors


%cp -av /content/drive/MyDrive/Lora /content/stable-diffusion-webui/models/Lora
%cp -av /content/drive/MyDrive/Checkpoint /content/stable-diffusion-webui/models/Stable-diffusion

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TNitro/ui-config/resolve/main/ui-config.json -d /content/stable-diffusion-webui -o ui-config.json
!sed -i -e 's/"txt2img\/Negative prompt\/value": "",/"txt2img\/Negative prompt\/value": "(low quality, worst quality), EasyNegativeV2,((nudity)), (NSFW)",/g' /content/stable-diffusion-webui/ui-config.json

!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared_options.py
!python launch.py --listen --enable-insecure-extension-access --theme dark --gradio-queue --multiple #--xformers

In [ ]:
#@title เอาไว้ Run ใหม่ถ้าค้าง
%cd /content/stable-diffusion-webui
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared_options.py
!python launch.py --listen --enable-insecure-extension-access --theme dark --gradio-queue --multiple --autolaunch #--xformers